# **基于Bilibili UP主：[秋葉aaaki]发布的[保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1fs4y1x7p2/)**
最核心的文件的整合代码均由主要作者[秋葉aaaki](https://github.com/Akegarasu/lora-scripts)完成。
开始前**建议阅读**：
1.   [保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1AL411q7Ub/?spm_id_from=333.337.search-card.all.click&vd_source=e6294619bd5137ea0025a3d001e715a7)
2.   [参数心得](https://www.bilibili.com/video/BV1GM411E7vk/)
3.   [训练教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
4.   [阿里云部署教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
> 本人只是完成阿里云下的依赖安装，使用的下载源不保证长期有效。本人对python与linux的使用能力有限，代码在各位大佬的指导和WSH032大佬于colab的依赖代码修改下完成，不足部分，有兴趣者可修改并分享。

> *--分享的责任与获取的自由*
**待解决问题**:

> 1.关于数据集的挂载免费领用的教程，我这边没有相应教程如有需要可自行解决

> 2.总是有人说觉得阿里云的环境有问题比本地更容易欠拟或者过拟，不知道是少了哪一步。

更新内容：
可以sdxl训练了
已经完成了秋叶于阿里云的web部署，金字塔噪声，分层训练，梯度都可以使用了
神童优化器可以使用了
采用了群友指月的修复代码，现已解决了张量版

根据其它群友的意见对代码进行了升级

训练包版本为2023年8月12日秋叶最新版

已突破抱脸封锁，所有文件均不使用抱脸下载，不会再出现下载失败的情况了，

## 一、基础依赖和训练包拉取(时间会更长)

In [ ]:
#查看是什么GPU
!nvidia-smi
#安装解压软件和下载存储工具
!apt-get update  > /dev/null 2>&1 
!apt-get install zip -y > /dev/null 2>&1
!apt install -y aria2 > /dev/null 2>&1 && echo 全部成功

In [ ]:
%cd  /mnt/workspace/
!git clone -b v8.15qing  https://ghproxy.com/https://github.com/taiczhi/lora-scripts.git
!cd /mnt/workspace/lora-scripts/huggingface/ && wget -c https://liblibai-online.vibrou.com/web/model/c52a9848ee316e16b59a0fe17e17edc4c767f97f652b439d9da758d6077160ae.ckpt  -O 1.zip
!unzip /mnt/workspace/lora-scripts/huggingface/1.zip -d /mnt/workspace/lora-scripts/huggingface/ > /dev/null 2>&1 && echo 解压成功
!sudo rm -r /mnt/workspace/lora-scripts/huggingface/1.zip   > /dev/null 2>&1  && echo 删除压缩包成功
!mkdir -p /mnt/workspace/lora-scripts/train/aki
!mkdir -p /mnt/workspace/lora-scripts/train/reg

In [ ]:
#安装依赖
!pip -q install torch==2.0.1 torchvision xformers triton 
#进入目录
%cd /mnt/workspace/lora-scripts/sd-scripts/
#安装目录要求依赖
!pip -q  install --upgrade -r requirements.txt && echo 安装需求依赖成功
#安装训练指示器
!pip -q install --upgrade lion-pytorch lycoris-lora dadaptation prodigyopt fastapi uvicorn wandb  && echo 安装训练器成功
!pip install --upgrade --no-deps pytorch-optimizer -i https://mirror.baidu.com/pypi/simple
#进行bitsandbytes的重装
!pip uninstall -q bitsandbytes -y > /dev/null 2>&1 && echo 卸载成功
!pip install bitsandbytes==0.41.1 --no-deps --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui --force > /dev/null 2>&1    && echo 安装成功
!pip uninstall -y tensorboard tb-nightly > /dev/null 2>&1 echo 卸载成功
!pip install tensorboard > /dev/null 2>&1 echo 安装成功
!pip install tb-nightly  > /dev/null 2>&1 echo 安装成功 /dev/null 2>&1 echo 所有依赖全部安装成功

## 二、模型下载（可自行改变路径）

### 只修改下方链接即可替换底膜  

默认底膜：是青龙sdxl1.0底膜base

In [ ]:
!cd /mnt/workspace/lora-scripts/sd-models/ && wget -c  https://liblibai-online.vibrou.com/web/model/1fa5725f4fe0a5fae0a840ea23725e3381213951e012aa39c3c9b70882c57015.safetensors -O down.safetensors && echo 下载成功 #下载底部模型

#### 常用模型下载
默认底膜：Chilloutmix-Ni-pruned-fp32-fix
https://liblibai-online.vibrou.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors

其它底膜链接：

https://liblibai-online.vibrou.com/web/model/e4a30e4607faeb06b5d590b2ed8e092690c631da0b2becb6224d4bb5327104b7.safetensors majicMIX realistic

https://liblibai-online.vibrou.com/web/model/e4a30e4607faeb06b5d590b2ed8e092690c631da0b2becb6224d4bb5327104b7.safetensors 麦橘v6版本模型

https://liblibai-online.vibrou.com/web/model/f8bb2922e1dc877dc0d33ed9b9dbbdba612b8b37711cbe17d803dbc92dd65b78.safetensors mix全能模型

https://liblibai-online.vibrou.com/web/model/c932413949a815fc64378ffa6ec34944597c5ff57cd98541347c4107106fd5d5.safetensors 娜乌斯嘉lora模型

官方sdxl模型base:  

https://www.wisemodel.cn/file-proxy/LiuDQ/stable-diffusion-xl-base-1.0-stabilityai/-/raw/main/sd_xl_base_1.0.safetensors?inline=false

模型路径:/mnt/workspace/lora-scripts/sd-models/

青龙修复底膜
https://liblibai-online.vibrou.com/web/model/1fa5725f4fe0a5fae0a840ea23725e3381213951e012aa39c3c9b70882c57015.safetensors

sd1.5基础模型：

https://www.wisemodel.cn/file-proxy/LiuDQ/stable-diffusion-v1.5-runwayml/-/raw/main/v1-5-pruned.safetensors?inline=false

模型路径:/mnt/workspace/lora-scripts/sd-models/

## 三、训练集下载（可自行改变路径）

### 只修改下方链接即可替换数据集

In [ ]:
!cd /mnt/workspace/lora-scripts/train/aki/  && wget -c        -O 1.zip > /dev/null 2>&1 && echo 下载成功 #下载训练集

In [ ]:
!unzip /mnt/workspace/lora-scripts/train/aki/1.zip -d /mnt/workspace/lora-scripts/train/aki/ > /dev/null 2>&1 && echo 解压成功   #解压训练集压缩包
!sudo rm -r /mnt/workspace/lora-scripts/train/aki/1.zip  > /dev/null 2>&1  && echo 删除压缩包成功 #删除训练集压缩包

## 四、启动青龙的一键式大模型训练界面

In [ ]:
%cd /mnt/workspace/lora-scripts    
!bash train_sdxldb.sh 

## 五、启动青龙的一键式lora训练界面

In [ ]:
%cd /mnt/workspace/lora-scripts    
!bash train_sdxllora.sh 